# **#1 Load dataset from GCP Cloud Storage**

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.cloud import storage

In [3]:
project_id = "onyx-sequencer-436913-i3"
client = storage.Client(project=project_id)

In [4]:
bucket_name = "cifake"
file_path = "cifake-real-and-ai-generated-synthetic-images.zip"
destination = "/content/cifake-real-and-ai-generated-synthetic-images.zip"

In [5]:
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(file_path)
blob.download_to_filename(destination)

In [6]:
import zipfile

with zipfile.ZipFile(destination, 'r') as zip_ref:
    zip_ref.extractall("/content/cifake-real-and-ai-generated-synthetic-images")

# **#2 Prepare Training and Test Sets**

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, metrics, callbacks, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

In [8]:
train_data = '/content/cifake-real-and-ai-generated-synthetic-images/train'
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_data,
  labels='inferred',
  validation_split=0.6,
  subset="training",
  seed=123,
  image_size=(225, 225),
  batch_size=1024)

Found 100000 files belonging to 2 classes.
Using 40000 files for training.


In [42]:
train_ds.class_names

['FAKE', 'REAL']

In [9]:
test_data = '/content/cifake-real-and-ai-generated-synthetic-images/test'
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_data,
  labels='inferred',
  validation_split=0.3,
  subset="validation",
  seed=123,
  image_size=(225, 225),
  batch_size=1024)

Found 20000 files belonging to 2 classes.
Using 6000 files for validation.


In [44]:
test_ds.class_names

['FAKE', 'REAL']

In [10]:
X_train = []
y_train = []

for images, labels in train_ds:
    X_train.append(images.numpy())
    y_train.append(labels.numpy())

X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train)

In [11]:
X_test = []
y_test = []

for images, labels in test_ds:
    X_test.append(images.numpy())
    y_test.append(labels.numpy())

X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test)

In [47]:
X_train.shape

(40000, 225, 225, 3)

In [48]:
y_train.shape

(40000,)

In [49]:
X_test.shape

(2000, 225, 225, 3)

In [50]:
y_test.shape

(2000,)

In [51]:
np.unique(y_train)
np.unique(y_test)

array([0, 1], dtype=int32)

# **#3 Normalise X_train and X_test**

In [12]:
X_train = X_train/255
X_test = X_test/255

# **#4 CNN Baseline Model**

In [13]:
model = Sequential()

model.add(layers.Conv2D(4, kernel_size=(5, 5), activation='relu', padding='valid', input_shape=(225, 225, 3)))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Conv2D(8, kernel_size=(5, 5), activation='relu', padding='valid', input_shape=(225, 225, 3)))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Conv2D(16, kernel_size=(5, 5), activation='relu', padding='valid', input_shape=(225, 225, 3)))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Conv2D(32, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

model.add(layers.Flatten())

model.add(layers.Dense(2, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 221, 221, 4)         │             304 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 110, 110, 4)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 106, 106, 8)         │             808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 53, 53, 8)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 49, 49, 16)          │           3,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 24, 24, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 22, 22, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 11, 11, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 3872)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2)                   │           7,746 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │               3 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 16,717 (65.30 KB)

 Trainable params: 16,717 (65.30 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [15]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,  # Use 20% of the training data for validation
    epochs=50,
    batch_size=2048,
    callbacks=[early_stopping]
)

Epoch 1/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 70s 3s/step - accuracy: 0.5540 - loss: 0.6857 - val_accuracy: 0.6194 - val_loss: 0.6438
Epoch 2/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 486ms/step - accuracy: 0.6495 - loss: 0.6282 - val_accuracy: 0.6898 - val_loss: 0.5731
Epoch 3/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 484ms/step - accuracy: 0.7060 - loss: 0.5668 - val_accuracy: 0.7245 - val_loss: 0.5433
Epoch 4/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 485ms/step - accuracy: 0.7223 - loss: 0.5402 - val_accuracy: 0.7467 - val_loss: 0.5127
Epoch 5/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 484ms/step - accuracy: 0.7445 - loss: 0.5113 - val_accuracy: 0.7556 - val_loss: 0.4919
Epoch 6/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 484ms/step - accuracy: 0.7665 - loss: 0.4821 - val_accuracy: 0.7738 - val_loss: 0.4696
Epoch 7/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 485ms/step - accuracy: 0.7793 - loss: 0.4616 - val_accuracy: 0.7665 - val_loss: 0.4661
Epoch 8/50
16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 483ms/step - accuracy: 0.7844 - loss: 0.4545 - val_accuracy: 0.79

In [16]:
model.evaluate(X_test, y_test,verbose=0)

[0.3990514874458313, 0.8224999904632568]

In [17]:
model.save('image_model_high_res_cnn.h5')